In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
from PIL import ImageFile

In [2]:
# Включение профилирования CUDNN
torch.backends.cudnn.benchmark = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [3]:
# Путь к данным animals
train_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\train'

test_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\test'

In [4]:
# Преобразования для тренировочного и валидационного наборов данных
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # Случайно применяет изменения яркости, контрастности, короче светокор))
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=(-10, 10)),
    # преобразует изображение в оттенки серого 
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.1, value='random'),
    # аффинное преобразование (смещение)
    transforms.RandomApply(torch.nn.ModuleList([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))]), p=0.25),
    # перспективное искажение
    transforms.RandomApply(torch.nn.ModuleList([transforms.RandomPerspective(distortion_scale=0.5, p=0.5)]), p=0.25),

    transforms.RandomErasing(p=0.25)
])

In [5]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
train_dataset = datasets.ImageFolder(train_path, transform=train_transform)

val_dataset = datasets.ImageFolder(test_path, transform=val_transform)

In [7]:
print(len(train_dataset.class_to_idx))

101


In [8]:
batch_size = 512

num_w = 12

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_w, pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_w, pin_memory=True)

In [9]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
# Размерность выходных признаков из модели CLIP
hidden_size = model.config.projection_dim

In [11]:
class CustomCLIPModel(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CustomCLIPModel, self).__init__()
        self.clip_model = clip_model
        self.fc1 = nn.Linear(hidden_size, 512)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.clip_model.get_image_features(x)
        x = self.fc1(features)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [12]:
num_classes = len(train_dataset.classes)
custom_model = CustomCLIPModel(model, num_classes).to(DEVICE)

#criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(custom_model.parameters(), lr=1e-3, weight_decay=1e-4) # 1e-4
#optimizer = optim.AdamW(custom_model.parameters(), lr=1e-4, weight_decay=1e-5)

#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [13]:
from torch.cuda.amp import autocast, GradScaler

In [14]:
num_epochs = 120
train_loss_history = []
val_accuracy_history = []
val_f1_history = []

scaler = torch.cuda.amp.GradScaler()

In [15]:
# Разморозка слоев модели CLIP для дообучения
for param in custom_model.clip_model.parameters():
    param.requires_grad = True

In [16]:
from sklearn.metrics import accuracy_score, f1_score

for epoch in range(num_epochs):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    custom_model.train()
    running_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')

    for inputs, labels in train_loader_tqdm:
        inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = custom_model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        train_loader_tqdm.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_dataset)
    train_loss_history.append(epoch_loss)

    # Валидация модели
    custom_model.eval()
    val_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)

            with torch.cuda.amp.autocast():
                outputs = custom_model(inputs)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())

    val_loss /= len(val_dataset)
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}, Val F1-Score: {f1:.4f}')
    #scheduler.step(accuracy)
    scheduler.step()

Epoch 1/120: 100%|██████████| 148/148 [01:37<00:00,  1.51it/s, loss=2.82]


Epoch 1/120 - Train Loss: 3.4599, Val Loss: 1.8106, Val Accuracy: 0.7439, Val F1-Score: 0.7366


Epoch 2/120: 100%|██████████| 148/148 [01:36<00:00,  1.53it/s, loss=2.46]


Epoch 2/120 - Train Loss: 2.6840, Val Loss: 1.5992, Val Accuracy: 0.7958, Val F1-Score: 0.7908


Epoch 3/120: 100%|██████████| 148/148 [01:36<00:00,  1.53it/s, loss=2.5] 


...

Epoch 118/120 - Train Loss: 2.1764, Val Loss: 1.2975, Val Accuracy: 0.8567, Val F1-Score: 0.8557


Epoch 119/120: 100%|██████████| 148/148 [01:34<00:00,  1.56it/s, loss=2.18]


Epoch 119/120 - Train Loss: 2.1894, Val Loss: 1.2962, Val Accuracy: 0.8592, Val F1-Score: 0.8586


Epoch 120/120: 100%|██████████| 148/148 [01:34<00:00,  1.57it/s, loss=2.13]


Epoch 120/120 - Train Loss: 2.1893, Val Loss: 1.2967, Val Accuracy: 0.8589, Val F1-Score: 0.8587
